## Dispersion relation animation

Based on the linear dispersion relation from equation 1.4.26 of Coastal and Estuarine Processes by Peter Nielsen. This animation allows the user to experiment with water depth 'h' and wavelength 'L' to see how that changes the celerity of a wave. The wave height slider has no influence on the celerity, it is simply there so the user can make the animation look more realistic for different depths. For this animation to work correctly, you need to have 4 images in the working directory, "deep_water_app.jpg", "full_eqn.jpg", "shallow_water_app.jpg" and "gray.jpg".

In [ ]:
# imported libraries
from tkinter import *
import matplotlib as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np
import math
import time
from scipy.signal import find_peaks
from PIL import ImageTk, Image
import cv2
plt.use("TkAgg")

"""%%%%%%%%%%%%%%%%%%% GUI CLASS %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""

# get images
show_image = cv2.imread("deep_water_app.jpg")
b, g, r = cv2.split(show_image)
dur_tracks_map = cv2.merge((r, g, b))
im_deep = Image.fromarray(dur_tracks_map)

# get images
show_image = cv2.imread("full_eqn.jpg")
b, g, r = cv2.split(show_image)
dur_tracks_map = cv2.merge((r, g, b))
im_full = Image.fromarray(dur_tracks_map)

# get images
show_image = cv2.imread("shallow_water_app.jpg")
b, g, r = cv2.split(show_image)
dur_tracks_map = cv2.merge((r, g, b))
im_shallow = Image.fromarray(dur_tracks_map)

# get images
show_image = cv2.imread("gray.jpg")
b, g, r = cv2.split(show_image)
dur_tracks_map = cv2.merge((r, g, b))
im_gray = Image.fromarray(dur_tracks_map)

class MainWindow:

    # master is same as root
    def __init__(self, master):

        # 'global-like' variables
        self.phase = 0
        self.h = 0.8 # range from 0 to 4.5m
        self.L = 15 # range from 1 to 70m
        self.Hw = 0.5 # range from 0.1 to 2.5m
        self.g = 9.79 # m/s^2 gravity
        self.T = ( (2*math.pi*self.L) /
                   (self.g*math.tanh((2*math.pi*self.h)/self.L))
                   )**0.5   # calculated from dispersion relation
        self.c = self.L/self.T

        self.L_limit = 70
        self.h_limit = 7
        self.h_limit_bound = 2.5

        # setup frames
        self.graph_frame = Frame(master)
        self.graph_frame.pack(side=LEFT)
        self.eqn2_frame = Frame(self.graph_frame)
        self.eqn2_frame.pack(side=BOTTOM)
        self.eqn_frame = Frame(self.graph_frame)
        self.eqn_frame.pack(side=BOTTOM)
        self.eqn_left_frame = Frame(self.eqn_frame)
        self.eqn_left_frame.pack(side=LEFT)
        self.eqn_mid_frame = Frame(self.eqn_frame)
        self.eqn_mid_frame.pack(side=LEFT)
        self.eqn_right_frame = Frame(self.eqn_frame)
        self.eqn_right_frame.pack(side=LEFT)
        self.side_2_frame = Frame(master)
        self.side_2_frame.pack(side=RIGHT, anchor=E)
        self.side_frame = Frame(master)
        self.side_frame.pack(side=RIGHT, anchor=E)

        # ######## CELERITY INIT #################
        self.gray_img = ImageTk.PhotoImage(image=im_gray)
        self.deep_img = ImageTk.PhotoImage(image=im_deep)
        self.deep_water = Label(self.eqn_left_frame, image=self.gray_img)
        self.deep_water.pack()

        self.full_img = ImageTk.PhotoImage(image=im_full)
        self.full_water = Label(self.eqn_mid_frame, image=self.full_img)
        self.full_water.pack()

        self.shallow_img = ImageTk.PhotoImage(image=im_shallow)
        self.shallow_water = Label(self.eqn_right_frame, image=self.gray_img)
        self.shallow_water.pack()

        self.celerity_period = Label(self.eqn2_frame,
                                     text="Celerity, c = " + '{0:.2f}'.format(self.c) + " m/s. Period, T = " + '{0:.2f}'.format(self.T) + " s.")
        self.celerity_period.config(font=("Courier", 18, "bold"))
        self.celerity_period.pack(side=BOTTOM)

        # ########## GRAPH INIT ################
        # https://datatofish.com/matplotlib-charts-tkinter-gui/ used this website to help write this section

        figure1 = Figure(figsize=(12, 4), dpi=100)
        self.ax1 = figure1.add_subplot(111)

        self.wave_x = np.linspace(0, self.L_limit, self.L_limit * 5)
        self.wave_y = self.Hw/2 * np.sin((2 * math.pi) / self.L * self.wave_x + self.phase) + self.h

        self.canvas = FigureCanvasTkAgg(figure1, master=self.graph_frame)
        self.canvas.get_tk_widget().pack(side=BOTTOM)

        # ######## HEADING INIT #################
        self.heading = Label(self.graph_frame, text="Celerity from the Dispersion Relation")
        self.heading.config(font=("Courier", 24, "bold"))
        self.heading.pack(side=TOP)

        # ######## SLIDER INIT 2 ##################
        self.graph_title_L = Label(self.graph_frame, text="L (m)")
        self.graph_title_L.config(font=("Courier", 18, "bold"), foreground="red")
        self.graph_title_L.pack(side=BOTTOM)

        L_start = 1
        L_end = self.L_limit
        self.graph_slider_L = Scale(self.graph_frame, from_=L_start, to=L_end,
                                  orient=HORIZONTAL, resolution=1, command=self.sliderUpdateL)
        self.graph_slider_L.set(self.L)
        self.graph_slider_L.config(font=("Courier", 18))
        self.graph_slider_L.pack(side=BOTTOM, fill=BOTH)

        # ######## SLIDER INIT ##################
        h_start = 0.1
        h_end = self.h_limit - self.h_limit_bound
        self.graph_slider_h = Scale(self.side_frame, from_=h_end, to=h_start, length=600,
                                    orient=VERTICAL, resolution=0.1, command=self.sliderUpdateh)
        self.graph_slider_h.set(self.h)
        self.graph_slider_h.config(font=("Courier", 18))
        self.graph_slider_h.pack(side=TOP, fill=BOTH)

        self.graph_title_h = Label(self.side_frame, text="h(m)")
        self.graph_title_h.config(font=("Courier", 18, "bold"), foreground="#2A96FF")
        self.graph_title_h.pack(side=BOTTOM)

        # ######## SLIDER INIT 3 ##################
        Hw_start = 0.1
        Hw_end = self.h_limit_bound
        self.graph_slider_Hw = Scale(self.side_2_frame, from_=Hw_end, to=Hw_start, length=600,
                                       orient=VERTICAL, resolution=0.1, command=self.sliderUpdateHw)
        self.graph_slider_Hw.set(self.Hw)
        self.graph_slider_Hw.config(font=("Courier", 18))
        self.graph_slider_Hw.pack(side=TOP, fill=BOTH)

        self.graph_title_Hw = Label(self.side_2_frame, text="H(m)")
        self.graph_title_Hw.config(font=("Courier", 18))
        self.graph_title_Hw.pack(side=BOTTOM)


    def plotGraph(self, phase_i):

        self.ax1.clear()
        self.phase = phase_i

        # scene customisation
        self.ax1.plot([0, self.L_limit], [self.h_limit, -1 * self.h_limit_bound],
                      color='w')  # makes sure the axis range is correct

        self.ax1.plot(self.wave_x, np.zeros_like(self.wave_x), color=(0.95, 0.7, 0), marker='o',
                      markersize=0.5)  # sandy bottom
        self.ax1.scatter(
            np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]) / 160 * self.L_limit, np.array(
                [-1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, -1, -0.5, -1]) / 2.5 * self.h_limit_bound,
            s=3,
            color=(0.95, 0.7, 0))  # sand particles - just for fun!
        self.ax1.plot(
            [0, 0.25, 0.5, 0.25, 0.25, 0, 0.5, 0.25, 0.25, 0.25, 0.11, 0.05, 0.11, 0.25, 0.39, 0.45, 0.39, 0.25],
            [0, 0.6, 0, 0.6, 1.0, 1.0, 1.0, 1.0, 1.4, 1.4, 1.46, 1.6, 1.74, 1.8, 1.74, 1.6, 1.46, 1.4],
            color='k')  # to scale human body - just for fun! 1.8m

        # plot still water line
        self.wave_x = np.linspace(0, self.L_limit, self.L_limit * 5)
        self.ax1.plot(self.wave_x, self.h * np.ones_like(self.wave_x), 'b--',
                      markersize=0.25, linewidth=1)  # sandy bottom

        # create sin wave of width L_limit
        self.wave_y = self.Hw/2 * np.sin((2 * math.pi) / self.L * self.wave_x + self.phase) + self.h
        self.ax1.plot(self.wave_x, self.wave_y, color=(0, 0, 1), marker='o', markersize=0.5)

        peaks, _ = find_peaks(self.wave_y, height=0)
        self.ax1.scatter( self.wave_x[peaks], self.wave_y[peaks], s=150, color='r', marker= '|') # Points to visualise celerity

        self.ax1.set_xlabel("Distance, x (m)")
        self.ax1.set_ylabel("Depth, h (m)")
        self.canvas.draw()


    def sliderUpdateh(self, event):
        # get the slider value when changed and update the graph cursor. Also update the image
        # Update the graph cursor
        self.h = float(event)
        self.dispersionApplication()
        self.plotGraph(self.phase)

    def sliderUpdateL(self, event):
        # get the slider value when changed and update the graph cursor. Also update the image
        # Update the graph cursor
        self.L = float(event)
        self.dispersionApplication()
        self.plotGraph(self.phase)

    def sliderUpdateHw(self, event):
        # get the slider value when changed and update the graph cursor. Also update the image
        # Update the graph cursor
        self.Hw = float(event)
        self.dispersionApplication()
        self.plotGraph(self.phase)

    def dispersionApplication(self):
        # apply dispersion formula, if it doesn't work catch the error and print
        try:
            self.T = ((2 * math.pi * self.L) /
                      (self.g * math.tanh((2 * math.pi * self.h) / self.L))
                      ) ** 0.5  # calculated from dispersion relation

            if isinstance(self.T, complex):
                print("complex number as answer")
                self.T = 0
                self.c = 0
                self.celerity_period.config(text="Celerity, c = " + " complex" + ". Period, T = " + " complex.")
            else:
                self.c = self.L / self.T
                self.celerity_period.config(text="Celerity, c = " + '{0:.2f}'.format(self.c) + " m/s. Period, T = " + '{0:.2f}'.format(self.T) + " s.")

        except:
            print("calculation invalid")
            self.T = 0
            self.c = 0
            self.celerity_period.config(text="Celerity, c = " + "invalid" + ". Period, T = " + "invalid.")

        if self.h/self.L >= 0.5:
            self.deepWater()
        elif self.h/self.L <= 0.05:
            self.shallowWater()
        else:
            self.full()

    def deepWater(self):

        self.deep_water.config(image=self.deep_img)
        self.full_water.config(image=self.full_img)
        self.shallow_water.config(image=self.gray_img)

    def full(self):

        self.deep_water.config(image=self.gray_img)
        self.full_water.config(image=self.full_img)
        self.shallow_water.config(image=self.gray_img)

    def shallowWater(self):

        self.deep_water.config(image=self.gray_img)
        self.full_water.config(image=self.full_img)
        self.shallow_water.config(image=self.shallow_img)

phase_coef_i = 0.05

if __name__=="__main__":
    GUI_root = Tk()
    a = MainWindow(GUI_root)

    phase = 0

    def repeat(phase):

        previous_time = time.time()

        # I found at a wavelength of 7m there is a nice speed,
        # so the step size is determined by the wavelength.
        # Works fine since phase_coef is used to calculate phase difference and time delay.
        phase_coef = phase_coef_i*(1/a.L)*7

        phase = phase + phase_coef*2*math.pi

        a.plotGraph(phase)
        # This print line can be use to debug the true speed of the wave in the animation,
        # sometimes this doesn't work too well if your computer isn't fast enough.
        #print(str((time.time() - previous_time)*1000) + " vs " + str(round(a.T * 1000 * phase_coef)))
        GUI_root.after(round((a.T)*1000*phase_coef) - round((time.time() - previous_time)*1000), repeat, phase)

    repeat(phase)

    GUI_root.mainloop()
    
print("exited program")